In [65]:
!pip install redis
import redis

# Connect
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

## Number of fields in a hash record


In [66]:
a=r.hlen("actor:632")
print(a)

3


## How many actors and movies are stored in Redis?

In [67]:
len(r.keys('actor:*'))

1319

## List 5 actors born before 1980

In [69]:

actors = []
for key in r.scan_iter(match='actor:*'):
    result = r.hget(key, 'date_of_birth')
    if result is None:
        continue
    
    year=int(result)

    if year < 1980:
        actors.append({
            'id': key.split(':')[1],
            'first_name': r.hget(key, 'first_name'),
            'last_name':  r.hget(key, 'last_name'),
            'result': year
        })
    if len(actors) >= 5:
        break


for a in actors:
    print(f"{a['first_name']} {a['last_name']}  – {a['result']}")


Bruce Greenwood  – 1956
Darren Goldstein  – 1974
Corey Stoll  – 1976
Tony Way  – 1978
Russell Brand  – 1975


## Retrieve the genre and rating of the movie "The Imitation Game".

In [75]:

for key in r.scan_iter(match='movie:*'):
    moovie_title = r.hget(key, 'title')
    if moovie_title== "The Imitation Game":
       result=[]
       result =  r.hmget(key, ['genre', 'rating'])
      

print(key)       
print(result)

movie:422
['Biography', '8.1']


## List the top 5 highest-rated movies.

In [ ]:

moovies = []
for key in r.scan_iter(match='movie:*'):
    title = r.hget(key, 'title')
    rating = r.hget(key, 'rating')
    float_rating=float(rating)
    
    if rating  is None:
        continue


    moovies.append ({'key': key,
                 'tiltle': title,
                 'rating': float_rating})

top5 = sorted(moovies, key=lambda m: m['rating'], reverse=True)[:5]


print(top5)

[{'key': 'movie:787', 'tiltle': 'Vegas (doc)', 'rating': 9.4}, {'key': 'movie:521', 'tiltle': 'Boy 9', 'rating': 9.4}, {'key': 'movie:314', 'tiltle': 'The Shawshank Redemption', 'rating': 9.3}, {'key': 'movie:723', 'tiltle': 'Ween Live in Chicago', 'rating': 9.2}, {'key': 'movie:1025', 'tiltle': 'Over Canada: An Aerial Adventure', 'rating': 9.1}]


## How many movies have a rating above 7.5?

In [82]:

a = 0
for key in r.scan_iter(match='movie:*'):
    rating = r.hget(key, 'rating')
    if result is None:
        continue
    
    float_rating=float(rating)

    if float_rating > 7.5:
        a=a+1
       
   


print(a)


183


## Update the rating of the movie "The Imitation Game" to 8.5

In [87]:

for key in r.scan_iter(match='movie:*'):
    moovie_title = r.hget(key, 'title')
    if moovie_title== "The Imitation Game":
       a =r.hset(key, 'rating',8.5)
       result =  r.hmget(key, ['title', 'rating'])
      

    
print(result)

['The Imitation Game', '8.5']


## Add a new actor: "Zendaya", born in 1996.

In [127]:
actor_id = r.incr('actor:id:counter')
key = f'actor:{actor_id}'


r.hset(key, mapping={
    'first_name': 'Zendaya',
    'last_name': '',            
    'date_of_birth': '1996'
})


print(r.hgetall(key))

{'first_name': 'Zendaya', 'last_name': '', 'date_of_birth': '1996'}


## Delete the movie with title "The Room".

In [132]:
for key in r.scan_iter(match='movie:*'):
    moovie_title = r.hget(key, 'title')
    if moovie_title== "The Room":
        r.hdel(key)
    
    